In [133]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [134]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [135]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [136]:
%%capture pwd
!pwd

In [137]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-has-covid-19-affected-wealth-disparities-among-ethnic-groups-in-the-uk',
 300,
 500)

# Fig 1

In [138]:
df = pd.read_excel("raw/r7hhwealthethnicityfinal.xlsx", sheet_name='Table 3',skiprows=3)
df['HRP Ethnicity']=df['HRP Ethnicity'].str.replace(' background','')
df.columns=['Ethnicity','v','v25','v975','p']
df=df.sort_values(by='v')

In [139]:
f = "fig1_wealth"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Ethnicity,v,v25,v975,p
0,Other Asian,-280500,-341600,-221600,Statistically significant
9,Pakistani,-243700,-311700,-176200,Statistically significant
1,Bangladeshi,-201500,-292400,-105200,Statistically significant
2,Black African or Other Black,-153700,-201900,-103900,Statistically significant
5,Indian,-143100,-221200,-68300,Statistically significant


In [140]:
base = alt.Chart(f1).encode(
    y=alt.Y(
        "Ethnicity:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        )
    )
)
bar = base.mark_bar(color=colors["eco-turquiose"]).encode(
    x=alt.X(
        "v:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            # titleAlign="center",
            # titleAnchor="middle",
            title="estimated difference in total net household wealth (£)",
            titleY=25,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
errors = base.mark_rule(color=scale_lightness(colors["eco-turquiose"],0.6),strokeWidth=1).encode(
    x="v25:Q",
    x2="v975:Q"
)
points1 = base.mark_tick(color=scale_lightness(colors["eco-turquiose"],0.6),strokeWidth=1,size=7).encode(
    x="v25:Q"
)
points2 = points1.encode(
    x="v975:Q"
)
layer1 = (
    ((bar+errors+points1+points2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [145]:
df = pd.read_excel("raw/r7hhwealthethnicityfinal.xlsx", sheet_name='Table 1',skiprows=3,nrows=11).drop('Notes',axis=1)
df['Other wealth']=df['Total net wealth (£)']-df['Net financial wealth (£)']-df['Net property wealth (£)']-\
       df['Physical wealth (£)']-df['Private pension wealth (£)']
df.columns=[i.replace(' (£)','') for i in df.columns]
df['HRP Ethnicity']=df['HRP Ethnicity'].str.replace(' background','')
df=df[['HRP Ethnicity',
 'Net financial wealth',
 'Net property wealth',
 'Physical wealth',
 'Private pension wealth',
 'Other wealth']]
df=df.set_index('HRP Ethnicity').stack().reset_index()
df.columns=['Ethnicity','Wealth','value']

In [147]:
f = "fig2_wealth_categories"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Ethnicity,Wealth,value
0,White British,Net financial wealth,10000
1,White British,Net property wealth,123000
2,White British,Physical wealth,40800
3,White British,Private pension wealth,84800
4,White British,Other wealth,73700


In [148]:
open('visualisation/fig2/wealth_categories.json','w').write(json.dumps(list(df.T.to_dict().values())))

4289